### Complex Schema Validation with Avro
**Description**: Implement a solution in Python to validate records against a complex nested Avro schema.

Eg., Complex schema ( nested_schema.avsc ):

**Steps**:
1. Load schema
2. Example data to validate
3. Validate against schema
4. Read back to check

In [6]:
import json
from fastavro import parse_schema, writer, reader, validation

# Step 1: Define the nested Avro schema
schema_json = {
    "namespace": "example.avro",
    "type": "record",
    "name": "Employee",
    "fields": [
        {"name": "name", "type": "string"},
        {"name": "age", "type": "int"},
        {"name": "address", "type": {
            "type": "record",
            "name": "Address",
            "fields": [
                {"name": "street", "type": "string"},
                {"name": "city", "type": "string"},
                {"name": "zip", "type": "string"}
            ]
        }},
        {"name": "skills", "type": {"type": "array", "items": "string"}}
    ]
}

# Step 2: Parse schema
parsed_schema = parse_schema(schema_json)

# Step 3: Define example data records
valid_record = {
    "name": "Alice",
    "age": 30,
    "address": {
        "street": "123 Main St",
        "city": "Springfield",
        "zip": "12345"
    },
    "skills": ["Python", "SQL", "Spark"]
}

invalid_record = {
    "name": "Bob",
    "age": "thirty",  # Invalid: should be int
    "address": {
        "street": "456 Elm St",
        "city": "Shelbyville",
        "zip": "67890"
    },
    "skills": ["Java", "C++"]
}

# Step 4: Validate records
print("Validating records:")

records = [valid_record, invalid_record]
valid_records = []

for idx, record in enumerate(records, start=1):
    try:
        validation.validate(record, parsed_schema)
        print(f" Record {idx} is valid? True")
        valid_records.append(record)
    except ValueError as e:
        print(f" Record {idx} is valid? False")
        print(f"  ValidationError: {e}")

# Step 5: Write only valid records to Avro file
avro_file = "employees.avro"
with open(avro_file, "wb") as out:
    writer(out, parsed_schema, valid_records)

print(f"\nWritten {len(valid_records)} valid record(s) to {avro_file}")

# Step 6: Read back valid records from Avro
print("\nReading back records from Avro:")
with open(avro_file, "rb") as fo:
    for record in reader(fo):
        print(record)


Validating records:
 Record 1 is valid? True


ValidationError: [
  "example.avro.Employee.age is <thirty> of type <class 'str'> expected int"
]